Projeto Final de Spark

Campanha Nacional de Vacinação contra Covid-19

Nível Avançado:
Replicar as visualizações do site “https://covid.saude.gov.br/”, porém acessando diretamente a API de Elastic.

In [48]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
# Criação da string com a URL de destino da API Elastic para fonte de dados MSCOVIDBR
es_url = "https://imunizacao-es.saude.gov.br/desc-imunizacao/_search"
# Agora vamos ler os dados do arquivo csv com o seguinte parâmetro da API:
# {
#  "size": 10000
# }
mscovidbrinputDf = spark.read.option('header', 'true').csv('/user/jes/data/mscovidbr_input.csv')
# E criamos uma tabela temporária agora usando o 'mscovidbrinputDf'
mscovidbrinputDf.createOrReplaceTempView('mscovidbrinputtbl')
# Agora criamos o mapa de parâmetros para passar para a fonte de dados REST
prmsMSCOVIDBR = { 'url' : es_url, 'input' : 'mscovidbrinputtbl', 'userId' : 'imunizacao_public', 'userPassword' : 'qlto5t&7r_@+#Tlstigi', 'method' : 'GET', 'readTimeout' : '10000', 'connectionTimeout' : '2000', 'partitions' : '10'}
# Agora criamos o Dataframe que contém o resultado da chamada para a API com o parâmetro
mscovidbrDf = spark.read.format("org.apache.dsext.spark.datasource.rest.RestDataSource").options(**prmsMSCOVIDBR).load()
# Nós inspecionamos a estrutura dos resultados retornados.
mscovidbrDf.printSchema()
# Criamos o Dataframe 'mscovidbrDf_vacina' utilizando a função 'explode' 
# para retornar uma nova linha para cada elemento no array com dados dos pacientes vacinados.
mscovidbrDf_vacina = mscovidbrDf.select(F.explode(F.col("output.hits.hits").alias("paciente")))
# Nós inspecionamos a estrutura dos resultados retornados.
mscovidbrDf_vacina.printSchema()
# Agora estamos prontos para aplicar SQL ou qualquer outro processamento nos resultados
# Visualiações:
# 1ª - Dos dados de vacinados por UF, município, sexo e idade(média, menor e maior)
mscovidbrDf_vacina.select(F.col("col._source.paciente_idade").alias("idade"), \
                    F.col("col._source.vacina_nome").alias("vacina"), \
                    F.col("col._source.vacina_descricao_dose").alias("dose")).show(truncate=False)
mscovidbrDf_vacina_sexo_idade = mscovidbrDf_vacina.groupBy(F.col("col._source.estabelecimento_uf").alias("UF"), \
                                                   F.col("col._source.estabelecimento_municipio_nome").alias("Municipio"), \
                                                   F.col("col._source.paciente_enumSexoBiologico").alias("Sexo") \
                                                   ).agg(F.format_number(F.avg(F.col("col._source.paciente_idade")),2).alias("Media_de_idade"), \
                                                       F.min(F.col("col._source.paciente_idade")).alias("Idade_menor"), \
                                                       F.max(F.col("col._source.paciente_idade")).alias("Idade_maior") \
                                                        ).sort(F.asc("UF"))
mscovidbrDf_vacina_sexo_idade.show(truncate=False)
# Salvando a primeira visualização como tabela Hive
mscovidbrDf_vacina_sexo_idade.write.saveAsTable("vacina_sexo_idade")
# 2ª - Dos dados de vacinados por UF, município, vacina e dose aplicada
mscovidbrDf_vacina_dose = mscovidbrDf_vacina.groupBy(F.col("col._source.estabelecimento_uf").alias("UF"), \
                                                   F.col("col._source.estabelecimento_municipio_nome").alias("Municipio"), \
                                                   F.col("col._source.vacina_nome").alias("Vacina"), \
                                                   F.col("col._source.vacina_descricao_dose").alias("Dose") \
                                                   ).agg(F.count(F.col("col._source.vacina_descricao_dose")).alias("Qtd_aplicacoes") \
                                                        ).sort(F.asc("UF"))
mscovidbrDf_vacina_dose.show(truncate=False)
# Salvando a segunda visualização com formato parquet e compressão snappy
mscovidbrDf_vacina_dose.write.save("/user/jes/vacina_parquet")
!hdfs dfs -ls /user/jes/vacina_parquet
# 3ª - Dos dados de vacinados por UF, município, raça, sexo, vacina e dose aplicada
mscovidbrDf_vacina_dose_sexoraca = mscovidbrDf_vacina.groupBy(F.col("col._source.estabelecimento_uf").alias("UF"), \
                                                   F.col("col._source.estabelecimento_municipio_nome").alias("Municipio"), \
                                                   F.col("col._source.vacina_nome").alias("Vacina"), \
                                                   F.col("col._source.vacina_descricao_dose").alias("Dose"), \
                                                   F.col("col._source.paciente_enumSexoBiologico").alias("Sexo"), \
                                                   F.col("col._source.paciente_racaCor_valor").alias("Raca") \
                                                   ).agg(F.count(F.col("col._source.paciente_racaCor_valor")).alias("Qtd_aplicacoes"), \
                                                         F.min(F.col("col._source.paciente_idade")).alias("Idade_menor"), \
                                                         F.max(F.col("col._source.paciente_idade")).alias("Idade_maior") \
                                                        ).sort(F.asc("UF"))
mscovidbrDf_vacina_dose_sexoraca.show()
# 4ª - Da quantidade de aplicações por vacina
mscovidbrDf_vacina_dose_aplicacoes = mscovidbrDf_vacina.groupBy(F.col("col._source.vacina_nome").alias("key") \
                                                   ).agg(F.count(F.col("col._source.vacina_nome")).cast("string").alias("value") \
                                                        )
mscovidbrDf_vacina_dose_aplicacoes.show(truncate=False)
mscovidbrDf_vacina_dose_aplicacoes.printSchema()
# Salvando a quarta visualização em um tópico no Kafka
mscovidbrDf_vacina_dose_aplicacoes.write \
                                  .format("kafka") \
                                  .option("kafka.bootstrap.servers", "kafka:9092") \
                                  .option("topic", "topic-kvvacina") \
                                  .save()

+-----------------------------------+-----+
|key                                |value|
+-----------------------------------+-----+
|Covid-19-Coronavac-Sinovac/Butantan|9083 |
|Vacina Covid-19 - Covishield       |917  |
+-----------------------------------+-----+

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = false)



Referência:

Neste projeto foi utilizada a biblioteca "Rest Data Source for Apache Spark" (https://github.com/sourav-mazumder/Data-Science-Extensions/tree/master/spark-datasource-rest)

Esta é uma biblioteca para chamar serviços/APIs baseados em REST para vários conjuntos de parâmetros de entrada em paralelo e agrupar os resultados, retornados pelo serviço REST, em um Dataframe.